In [1]:
from pyomo.environ import *
import pandas as pd

In [2]:
dem = pd.read_excel('C:/Jupyter/TESIS/instancias/instancia1.0.xlsx', sheet_name='dem')
cap = pd.read_excel('C:/Jupyter/TESIS/instancias/instancia1.0.xlsx', sheet_name='cap')
preco = pd.read_excel('C:/Jupyter/TESIS/instancias/instancia1.0.xlsx', sheet_name='precos')

In [3]:
dem_cor = {}
for i in range(dem.shape[0]):
    dem_cor[(dem.iloc[i,0], dem.iloc[i,1], dem.iloc[i,2])] = dem.iloc[i,3]

cap_cor = {}
for i in range(cap.shape[0]):
    cap_cor[cap.iloc[i,0]] = cap.iloc[i,1]

preco_cor = {}
for i in range(preco.shape[0]):
    preco_cor[(preco.iloc[i,0], preco.iloc[i,1], preco.iloc[i,2])] = preco.iloc[i,3]

In [4]:
n = 4
dem_cor_1 = {(i,j,k): dem_cor[(i,j,k)] if (i,j,k) in dem_cor else 0  for i in range(1, n+1) for j in range(1, n+1) for k in range(1, 2+1)}
preco_cor_1 = {(i,j,c): preco_cor[(i,j,c)] if (i,j,c) in preco_cor else 0  for i in range(1, n+1) for j in range(1, n+1) for c in range(1, 2+1)}

In [5]:
# instanciar o modelo
model = AbstractModel()

n = 4
# Escalares
model.n = Param(initialize = n)
model.i = Param(initialize = n)
model.j = Param(initialize = n)
model.k = Param(initialize = 2)

# conjuntos
model.I = RangeSet(1, model.i)
model.J = RangeSet(1, model.j)
model.K = RangeSet(1, model.k)

#parâmetros indexados
model.P = Param(model.I, model.J, model.K, initialize = preco_cor_1)
model.d = Param(model.I, model.J, model.K, initialize = dem_cor_1)
model.Q = Param(model.K, initialize = cap_cor)

# variáveis de decisão
model.X = Var(RangeSet(0, model.i), model.J, model.K, within = NonNegativeIntegers)
model.Y = Var(RangeSet(0, model.i), model.J, model.K, within = NonNegativeIntegers)
model.cap = Var(RangeSet(0, model.i), model.K, within = NonNegativeIntegers)

# função objetivo
def fo1(model):
    return sum( model.P[i,j,k]*((model.X[i,j,k])) for i in model.I for j in model.J for k in model.K if i < j)
model.OBJ1 = Objective(rule = fo1, sense = maximize)

# restrições
def cap1(model, i, k):
    return model.cap[i,k] ==  model.cap[i-1,k] - sum(model.Y[i-1,j,k] + model.X[i-1,j,k] for j in model.J if j>=i) + sum(model.Y[j,i,k] + model.X[j,i,k] for j in model.J if j<i)
model.ConstrainCap1 = Constraint(model.I, model.K, rule = cap1)

def cap2(model, i, k):
    if i < model.n:
        return sum(model.Y[i,j,k] + model.X[i,j,k] for j in model.J if i<j) <= model.cap[i,k]
    else:
        return Constraint.Skip
model.ConstrainCap2 = Constraint(model.I, model.K, rule = cap2)

def authorization(model, i, j, k):
    if i < j:
        return model.Y[i,j,k] >= model.X[i,j,k]
    else:
        return Constraint.Skip
model.ConstrainAuthorization = Constraint(model.I, model.J, model.K, rule = authorization)

def classe(model, i, j, k):
    if i < j and k < model.k:
        return model.Y[i,j,k] <= model.Y[i,j,k+1]
    else:
        return Constraint.Skip
model.ConstrainClasse = Constraint(model.I, model.J, model.K, rule = classe)

def assignment(model, i, j, k):
    if i < j:
        return model.X[i,j,k] <= model.d[i,j,k]
    else:
        return Constraint.Skip
model.ConstrainAssignment = Constraint(model.I, model.J, model.K, rule = assignment)

def assignment0(model, j, k):
    return model.X[0,j,k] == 0
model.ConstrainAssignment0 = Constraint(model.J, model.K, rule = assignment0)

def authorization0(model, j, k):
    return model.Y[0,j,k] == 0
model.ConstrainAuthorization0 = Constraint(model.J, model.K, rule = authorization0)

def cap0(model,k):
    return model.cap[0,k] == model.Q[k]
model.ConstrainCap0 = Constraint(model.K,rule = cap0)

instance = model.create_instance()

opt = SolverFactory('glpk')

results = opt.solve(instance, tee=True)

# instance.pprint()
print(results)


c:\Users\Wilmer\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\Wilmer\AppData\Local\Temp\tmp64xjl95r.glpk.raw --wglp C:\Users\Wilmer\AppData\Local\Temp\tmpb1ea5jvv.glpk.glp
 --cpxlp C:\Users\Wilmer\AppData\Local\Temp\tmpx7kcoe1m.pyomo.lp
Reading problem data from 'C:\Users\Wilmer\AppData\Local\Temp\tmpx7kcoe1m.pyomo.lp'...
62 rows, 50 columns, 152 non-zeros
50 integer variables, none of which are binary
460 lines were read
Writing problem data to 'C:\Users\Wilmer\AppData\Local\Temp\tmpb1ea5jvv.glpk.glp'...
367 lines were written
GLPK Integer Optimizer, v4.65
62 rows, 50 columns, 152 non-zeros
50 integer variables, none of which are binary
Preprocessing...
30 rows, 30 columns, 98 non-zeros
30 integer variables, none of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 30
Solving LP relaxation...
GLPK Simplex Optimize

In [6]:
print('Valor da função objetivo: ', '\033[1m' + '\033[32m' + str(value(instance.OBJ1)) + '\033[0m')

print('')

# print('Valores das variáveis de decisão: ')
lista = []
for i in range(1,n+1):
    for j in range(1,n+1):
        for k in range(1,2+1):
            if i<j:
                # print(i,j,k, ' - ',value(instance.Y[i,j,k]), ' - ', value(instance.X[i,j,k]) )
                lista.append([ i, j, k,value(instance.P[i,j,k]), value(instance.d[i,j,k]) , value(instance.X[i,j,k]), value(instance.Y[i,j,k]) ])

a = pd.DataFrame(lista, columns=['origem','destino','classe','preco','demanda','Assignments','Authorizations'])
print(a)
print('')
print('Valores das capacidades em cada nó: ')

for i in range(1, n+1):
    print(i , ' - ', value(instance.cap[i,1]), ' - ', value(instance.cap[i,2]))


print('')
tabela_pivote = pd.pivot_table(a[a['classe']==1],  values='Authorizations', index=['origem','classe'], columns=['destino'], aggfunc='sum', margins=True, margins_name='Total')
tabela_pivote2 = pd.pivot_table(a[a['classe']==1],  values='Assignments', index=['origem','classe'], columns=['destino'], aggfunc='sum', margins=True, margins_name='Total')

tabela_pivote3 = pd.pivot_table(a[a['classe']==2],  values='Authorizations', index=['origem','classe'], columns=['destino'], aggfunc='sum', margins=True, margins_name='Total')
tabela_pivote4 = pd.pivot_table(a[a['classe']==2],  values='Assignments', index=['origem','classe'], columns=['destino'], aggfunc='sum', margins=True, margins_name='Total')

result_1 = tabela_pivote + tabela_pivote2
result_2 = tabela_pivote3 + tabela_pivote4

print(result_1, '\n \n', result_2)


Valor da função objetivo:  3088.0

    origem  destino  classe  preco  demanda  Assignments  Authorizations
0        1        2       1     20        5          5.0             5.0
1        1        2       2     22        2          2.0             5.0
2        1        3       1     45        9          9.0             9.0
3        1        3       2     54        5          5.0             9.0
4        1        4       1     75        4          4.0           152.0
5        1        4       2     94        7          7.0           208.0
6        2        3       1     25        8          8.0             8.0
7        2        3       2     32        6          6.0             8.0
8        2        4       1     55        5          5.0             5.0
9        2        4       2     72        2          2.0             5.0
10       3        4       1     30        6          6.0            20.0
11       3        4       2     40        8          8.0            20.0

Valores das cap

In [7]:
# a = pd.DataFrame(lista, columns=['i','j','k','Y','X'])
# a.to_excel('wwww.xlsx')